In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
!git clone https://github.com/xinntao/ESRGAN.git


Cloning into 'ESRGAN'...
remote: Enumerating objects: 225, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 225 (delta 15), reused 14 (delta 14), pack-reused 205 (from 1)
Receiving objects: 100% (225/225), 24.86 MiB | 39.16 MiB/s, done.
Resolving deltas: 100% (85/85), done.


In [6]:
%cd ESRGAN

/kaggle/working/ESRGAN/ESRGAN


In [7]:
!pip install basicsr facexlib gfpgan
!python setup.py develop


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 3.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata 

In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader
import glob

# Define transforms for LR and HR separately
lr_transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Low-resolution input
    transforms.ToTensor(),
])

hr_transform = transforms.Compose([
    transforms.Resize((256, 256)),  # High-resolution ground truth
    transforms.ToTensor(),
])

# Update MySRDataset to accept two transforms
class MySRDataset(Dataset):
    def __init__(self, lr_dir, hr_dir, lr_transform=None, hr_transform=None):
        self.lr_paths = sorted(glob.glob(os.path.join(lr_dir, '*.png')))
        self.hr_paths = sorted(glob.glob(os.path.join(hr_dir, '*.png')))
        self.lr_transform = lr_transform
        self.hr_transform = hr_transform

    def __len__(self):
        return len(self.lr_paths)

    def __getitem__(self, idx):
        lr_image = Image.open(self.lr_paths[idx]).convert("RGB")
        hr_image = Image.open(self.hr_paths[idx]).convert("RGB")

        if self.lr_transform:
            lr_image = self.lr_transform(lr_image)
        if self.hr_transform:
            hr_image = self.hr_transform(hr_image)

        return lr_image, hr_image

# Now create the dataset using both transforms
test_dataset = MySRDataset(
    '/kaggle/input/superres/image_super_resolution_dataset/scaling_4x/test/LR',
    '/kaggle/input/superres/image_super_resolution_dataset/scaling_4x/test/HR',
    lr_transform=lr_transform,
    hr_transform=hr_transform
)

# Create the DataLoader
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


In [29]:
import RRDBNet_arch as arch
import torch

# Instantiate the model
model = arch.RRDBNet(3, 3, 64, 23, gc=32)

model.load_state_dict(torch.load('/kaggle/input/weights/RRDB_ESRGAN_x4.pth'))

<All keys matched successfully>

In [30]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [31]:
from torchmetrics.image import PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
psnr_metric = PeakSignalNoiseRatio(data_range=1.0).to(device)
ssim_metric = StructuralSimilarityIndexMeasure(data_range=1.0).to(device)

In [32]:
model.eval()  # set model to evaluation mode

total_psnr = 0.0
total_ssim = 0.0
num_batches = 0

with torch.no_grad():  # no gradients needed during evaluation
    for lr, hr in test_loader:
        lr = lr.to(device)
        hr = hr.to(device)

        sr = model(lr)  # super-resolved output from model

        # Calculate PSNR and SSIM
        batch_psnr = psnr_metric(sr, hr).item()
        batch_ssim = ssim_metric(sr, hr).item()

        total_psnr += batch_psnr
        total_ssim += batch_ssim
        num_batches += 1

avg_psnr = total_psnr / num_batches
avg_ssim = total_ssim / num_batches

print(f"Test PSNR: {avg_psnr:.4f}")
print(f"Test SSIM: {avg_ssim:.4f}")


Test PSNR: 27.3863
Test SSIM: 0.7033


In [ ]:
import random
import matplotlib.pyplot as plt

# Randomly pick an index
random_idx = random.randint(0, len(test_dataset) - 1)

# Get the LR and HR image tensors from the dataset
lr_img, hr_img = test_dataset[random_idx]

# Put model in eval mode
model.eval()

# Move LR image to device and add batch dimension
input_img = lr_img.unsqueeze(0).to(device)

# Generate SR image
with torch.no_grad():
    sr_img = model(input_img)

# Move images back to CPU and remove batch dimension
lr_img = lr_img.cpu()
hr_img = hr_img.cpu()
sr_img = sr_img.squeeze(0).cpu()

# Visualization function (assuming you have one, or define your own)
def imshow(tensor, title=None):
    image = tensor.permute(1, 2, 0).numpy()
    plt.imshow(image)
    if title:
        plt.title(title)
    plt.axis('off')

plt.figure(figsize=(9, 3))

plt.subplot(1, 3, 1)
imshow(lr_img, title='LR (Input)')

plt.subplot(1, 3, 2)
imshow(sr_img, title='SR (Output)')

plt.subplot(1, 3, 3)
imshow(hr_img, title='HR (Ground Truth)')

plt.show()


In [65]:
import os
from PIL import Image
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from skimage.metrics import structural_similarity as compare_ssim

# Paths for your LR and HR images folders
lr_dir = '/kaggle/input/superres/image_super_resolution_dataset/scaling_4x/test/LR'
hr_dir = '/kaggle/input/superres/image_super_resolution_dataset/scaling_4x/test/HR'

image_names = sorted(os.listdir(lr_dir))

psnr_list = []
ssim_list = []

for img_name in image_names:
    # Load original LR and HR images
    lr_img = Image.open(os.path.join(lr_dir, img_name)).convert('RGB')
    hr_img = Image.open(os.path.join(hr_dir, img_name)).convert('RGB')
    
    # Resize LR image to 64x64 (if not already)
    lr_resized = lr_img.resize((64, 64), Image.BICUBIC)
    
    # Resize HR image to 256x256 (if not already)
    hr_resized = hr_img.resize((256, 256), Image.BICUBIC)
    
    # Now upscale LR image to HR size (256x256) using bicubic
    bicubic_img = lr_resized.resize((256, 256), Image.BICUBIC)
    
    # Convert to numpy float arrays [0,1] for metrics
    bicubic_np = np.array(bicubic_img).astype(np.float32) / 255.0
    hr_np = np.array(hr_resized).astype(np.float32) / 255.0
    
    # Calculate PSNR and SSIM
    psnr = compare_psnr(hr_np, bicubic_np, data_range=1.0)
    ssim = compare_ssim(hr_np, bicubic_np, multichannel=True, data_range=1.0, channel_axis=2)

    
    psnr_list.append(psnr)
    ssim_list.append(ssim)

avg_psnr = sum(psnr_list) / len(psnr_list)
avg_ssim = sum(ssim_list) / len(ssim_list)

print(f"Average PSNR (Bicubic): {avg_psnr:.4f}")
print(f"Average SSIM (Bicubic): {avg_ssim:.4f}")


Average PSNR (Bicubic): 25.7640
Average SSIM (Bicubic): 0.6608


In [3]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import glob
import os
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from skimage.metrics import structural_similarity as compare_ssim

# Define SRCNN Model (example)
class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=9, padding=4)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(32, 1, kernel_size=5, padding=2)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.conv3(x)
        return x


# Dataset with transforms
class MySRDataset(Dataset):
    def __init__(self, lr_dir, hr_dir, lr_transform=None, hr_transform=None):
        self.lr_paths = sorted(glob.glob(os.path.join(lr_dir, '*.png')))
        self.hr_paths = sorted(glob.glob(os.path.join(hr_dir, '*.png')))
        self.lr_transform = lr_transform
        self.hr_transform = hr_transform

    def __len__(self):
        return len(self.lr_paths)

    def __getitem__(self, idx):
        lr_img = Image.open(self.lr_paths[idx]).convert('RGB')
        hr_img = Image.open(self.hr_paths[idx]).convert('RGB')

        if self.lr_transform:
            lr_img = self.lr_transform(lr_img)
        if self.hr_transform:
            hr_img = self.hr_transform(hr_img)

        return lr_img, hr_img

# Transforms
lr_transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((64, 64)),  # Low-res size
    transforms.ToTensor(),
])

hr_transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((256, 256)),  # High-res size
    transforms.ToTensor(),
])

# Create dataset and dataloader
test_dataset = MySRDataset('/kaggle/input/superres/image_super_resolution_dataset/scaling_4x/test/LR', '/kaggle/input/superres/image_super_resolution_dataset/scaling_4x/test/HR', lr_transform, hr_transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Initialize SRCNN model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SRCNN().to(device)
model.load_state_dict(torch.load("/kaggle/input/srcnn/srcnn_x4.pth", map_location=device))
model.eval()

# For bicubic upscaling before SRCNN
bicubic_upscale = transforms.Resize((256, 256), interpolation=Image.BICUBIC)

psnr_list = []
ssim_list = []

with torch.no_grad():
    for lr, hr in test_loader:
        lr = lr.to(device)        # LR image tensor, 3x64x64
        hr = hr.to(device)        # HR image tensor, 3x256x256

        # Upscale LR to HR size using bicubic interpolation (before SRCNN)
        # Convert tensor to PIL Image
        lr_pil = transforms.ToPILImage()(lr.squeeze(0).cpu())
        bicubic_img = bicubic_upscale(lr_pil)

        # Convert back to tensor and to device
        bicubic_tensor = transforms.ToTensor()(bicubic_img).unsqueeze(0).to(device)

        # Pass bicubic upscaled image through SRCNN
        sr = model(bicubic_tensor)

        # Clamp to [0,1]
        sr = torch.clamp(sr, 0.0, 1.0)

        # Convert to numpy arrays for metrics
        sr_np = sr.squeeze(0).permute(1, 2, 0).cpu().numpy()
        hr_np = hr.squeeze(0).permute(1, 2, 0).cpu().numpy()

        # PSNR and SSIM expect pixel range [0,1]
        psnr = compare_psnr(hr_np, sr_np, data_range=1.0)
        ssim = compare_ssim(hr_np, sr_np, data_range=1.0, channel_axis=2)

        psnr_list.append(psnr)
        ssim_list.append(ssim)

avg_psnr = sum(psnr_list) / len(psnr_list)
avg_ssim = sum(ssim_list) / len(ssim_list)

print(f"Average PSNR: {avg_psnr:.4f}")
print(f"Average SSIM: {avg_ssim:.4f}")


Average PSNR: 27.5452
Average SSIM: 0.7262
